Complete the exercises below For **Assignment #13**.

Load the `ISLR2` and the `tidymodels` packages.

In [51]:
library(tidymodels) |> suppressPackageStartupMessages()
library(ISLR2) |> suppressPackageStartupMessages()

In this assignment we will use the `Default` dataset which includes the default status for credit card customers (`default` variable) in addition to each customer's:

1. credit card balance (`balance` variable),
1. student status (`student` variable), and,
1. income (`income` variable).

In [52]:
Default |> head()

default student balance   income   
1 No      No       729.5265 44361.625
2 No      Yes      817.1804 12106.135
3 No      No      1073.5492 31767.139
4 No      No       529.2506 35704.494
5 No      No       785.6559 38463.496
6 No      Yes      919.5885  7491.559

We will be modeling `default` with the customer features.

Before we begin let's count how many customers fall into each `default` category.

In [53]:
Default |> count(default)

default n   
1 No      9667
2 Yes      333

The data is quite imbalanced. This will be important to keep in mind when we evaluate the performance of our model later. 

Run the code below to create and training data from `Default`. We will use the "test" dataset at the end to get a final evaluation of our best model's accuracy.

In [54]:
set.seed(1234)
Default_split = Default |>
    initial_split(
        prop = 0.90,
        strata = default
    )

Default_train = training(Default_split)
Default_test = testing(Default_split)

Create a logistic regression model called `mod`. Set the engine to `glm` and the mode to `classification`. 

In [55]:
mod <- logistic_reg() |>
    set_engine("glm") |>
    set_mode("classification")

Our data is imbalanced. As such, a naive model that *always* predicts a customer to **not default** would be correct quite often. Let's start by calculating the "accuracy" of a naive model. This will be the baseline accuracy by which we evaluate other models.

In [56]:
# This code calculates the accuracy of a model that always predicts default to be "No"
Default_train |>
    mutate(
        .pred_naive = factor(
            'No',
            levels = c('No', 'Yes')
        )
    ) |>
    accuracy(
        truth = default,
        .pred_naive
    )

.metric  .estimator .estimate
1 accuracy binary     0.9665556

Let's use k-fold cross validation to evaluate the performance of a model where the outcome is `default` and the predictors are `income` and `balance`.

To start, use `vfold_cv` to generate 10 validation folds (i.e. set the `v` variable to 10). Set the `strata` argument to `default` so we preserve the distribution of `default` values in each fold.

Create your folds below and use `glimpse` to look at the output table. Call your output folds tables "folds".

In [57]:
set.seed(1234)
folds <- Default_train |>
    vfold_cv(
        v = 10,
        strata = default
    )

glimpse(folds)

Rows: 10
Columns: 2
$ splits <list> [<vfold_split[8100 x 900 x 9000 x 4]>], [<vfold_split[8100 x 9…
$ id     <chr> "Fold01", "Fold02", "Fold03", "Fold04", "Fold05", "Fold06", "Fo…


The code below fits a model to each of your 10 folds. `collect_metrics` finds the average of evaluation metrics for each of your ten models. 

In [58]:
mod |>
    fit_resamples(
        default ~ income + balance,
        folds
    ) |>
    collect_metrics()

.metric     .estimator mean       n  std_err     .config             
1 accuracy    binary     0.97400000 10 0.001464720 Preprocessor1_Model1
2 brier_class binary     0.02129948 10 0.001051008 Preprocessor1_Model1
3 roc_auc     binary     0.94979424 10 0.007368964 Preprocessor1_Model1

❓How does the model accuracy compare to the naive model from above?

The model accuracy is marginally more accurate than the naive model, but not meaningfully so.

Complete the cell below to evaluate a model also includes the `student` variable as as predictor.
1. use `default ~ income + balance + student` as the formula,
2. encode your `student` variable with `step_dummy`, and,
3. don't forget to `prep` your recipe!

In [59]:
rec = Default_train |>
    recipe(
        default ~ .
    ) |>
    step_dummy(student)

rec |>
    prep() |>
    juice() |>
    head()

mod |>
    fit_resamples(rec, folds) |>
    collect_metrics()

balance   income    default student_Yes
1  729.5265 44361.625 No      0          
2  785.6559 38463.496 No      0          
3  919.5885  7491.559 No      1          
4 1161.0579 37468.529 No      0          
5    0.0000 29275.268 No      0          
6    0.0000 21871.073 No      1

.metric     .estimator mean       n  std_err     .config             
1 accuracy    binary     0.97355556 10 0.001470328 Preprocessor1_Model1
2 brier_class binary     0.02125681 10 0.001061736 Preprocessor1_Model1
3 roc_auc     binary     0.94994760 10 0.007579688 Preprocessor1_Model1

❓Does it appear that the model that includes `student` improves upon the first model with only `income` and `balance` as predictors?

No, the inclusion of `student` in the model does not appear to improve its accuracy.

Finally, estimate the accuracy of an `default ~ income + balance` model on the test data, `Default_test`. 

In [60]:
test_data_predictions <- mod |>
    fit(
        default ~ income + balance,
        data = Default_train
    ) |>
    predict(
        new_data = Default_test
    )

table(test_data_predictions)

Default_test |>
    bind_cols(
        test_data_predictions
    ) |>
    accuracy(
        truth = default,
        estimate = .pred_class
    )

.pred_class
 No Yes 
989  11 

.metric  .estimator .estimate
1 accuracy binary     0.971

❓Does our model outperform a naive model?

Our model marginally outperforms the naive model (~97.1% vs ~96.6%), which would potentially identify ~1/3 of the defaulting customers in the original data.